In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jfleg-dataset/eval.csv
/kaggle/input/jfleg-dataset/train.csv


In [2]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
import datasets
import pandas as pd


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
checkpoint = "t5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/jfleg-dataset/train.csv")
test_def = pd.read_csv("/kaggle/input/jfleg-dataset/eval.csv")

combined_df = pd.concat([train_df, test_def], ignore_index=True)
combined_df.to_csv("/kaggle/working/combined.csv", index=False)
combined_df.head()

,input,target
0,So I think we can not live if old people could...,So I think we would not be alive if our ancest...
1,So I think we can not live if old people could...,So I think we could not live if older people d...
2,So I think we can not live if old people could...,So I think we can not live if old people could...
3,So I think we can not live if old people could...,So I think we can not live if old people can n...
4,For not use car.,Not for use with a car.


In [5]:
import json
# Convert combined_df to json format and save it.
data = {'data': []}

for i in range(len(combined_df)):
    json_data = {}
    json_data = {
        "input": "grammar: " + combined_df['input'][i],
        "target": combined_df['target'][i]
    }   
    data['data'].append(json_data)
with open('/kaggle/working/combined.json', 'w') as f:
    json.dump(data, f, indent=4, separators=(',', ': ') )

In [6]:
from datasets import load_dataset
dataset = load_dataset('json', data_files='/kaggle/working/combined.json',field="data")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ea24147818b0cc8f/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
dataset = dataset['train'].train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 4803
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 1201
    })
})

In [8]:
dataset['train'][0]

{'input': 'grammar: Or, " Is this the right thing to do? " ',
 'target': 'Or, " Is this the right thing to do? " '}

In [9]:
def preprocess_func(examples):
    inputs = [source for source in examples['input']]
    targets = [target for target in examples['target']]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=1024, truncation=True)
    return model_inputs

In [10]:
tokenized_data = dataset.map(preprocess_func, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [11]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4803
    })
    test: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1201
    })
})

In [12]:
!pip install evaluate sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/de/ea/025db0a39337b63d4728a900d262c39c3029b0fe76a9876ce6297b1aa6a0/sacrebleu-2.4.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 2.5 MB/s eta 0:00:00
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 11.0 MB/s eta 0:00:00


In [13]:
import evaluate
metric = evaluate.load("sacrebleu")

# Then create a function that passes your predictions and labels to compute to calculate the SacreBLEU score:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, max_new_tokens=1024)
#     print(f"Decoded preds: {decoded_preds} \n Len = {len(decoded_preds)}")
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [14]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [15]:

training_args = Seq2SeqTrainingArguments(
    output_dir="trainer-state",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=40,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    logging_dir="logging",
    logging_strategy="epoch",
#     generation_max_length=1024,
#     max_new_tokens=1024,  
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.712100,0.501570,50.857200,16.410500
2,0.553600,0.467730,51.967400,16.394700
3,0.503900,0.446720,52.628600,16.354700
4,0.466600,0.435286,52.820100,16.356400
5,0.435500,0.423996,53.141300,16.333100
6,0.406700,0.419589,53.291100,16.326400
7,0.390200,0.416209,53.657200,16.318900
8,0.368600,0.416930,53.667000,16.302200
9,0.353300,0.417876,53.542900,16.308900
10,0.342100,0.420589,53.400700,16.298900


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=12040, training_loss=0.30130216909009355, metrics={'train_runtime': 5255.5621, 'train_samples_per_second': 36.556, 'train_steps_per_second': 2.291, 'total_flos': 1.172250361930752e+16, 'train_loss': 0.30130216909009355, 'epoch': 40.0})

In [ ]:
trainer.save_model("gec-t5-base")

In [24]:
# zip model folder to download
!zip -r trained_model1.zip /kaggle/working/gec-t5-base

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


updating: kaggle/working/gec-t5-base/ (stored 0%)
updating: kaggle/working/gec-t5-base/config.json (deflated 63%)
updating: kaggle/working/gec-t5-base/generation_config.json (deflated 29%)
updating: kaggle/working/gec-t5-base/tokenizer.json (deflated 74%)
updating: kaggle/working/gec-t5-base/training_args.bin (deflated 49%)
updating: kaggle/working/gec-t5-base/spiece.model (deflated 48%)
updating: kaggle/working/gec-t5-base/tokenizer_config.json (deflated 95%)
updating: kaggle/working/gec-t5-base/model.safetensors (deflated 8%)
updating: kaggle/working/gec-t5-base/special_tokens_map.json (deflated 86%)


In [26]:
# zip model folder to download
!zip -r fine-tuned.zip /kaggle/working/gec-t5-base

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/gec-t5-base/ (stored 0%)
  adding: kaggle/working/gec-t5-base/config.json (deflated 63%)
  adding: kaggle/working/gec-t5-base/generation_config.json (deflated 29%)
  adding: kaggle/working/gec-t5-base/tokenizer.json (deflated 74%)
  adding: kaggle/working/gec-t5-base/training_args.bin (deflated 49%)
  adding: kaggle/working/gec-t5-base/spiece.model (deflated 48%)
  adding: kaggle/working/gec-t5-base/tokenizer_config.json (deflated 95%)
  adding: kaggle/working/gec-t5-base/model.safetensors (deflated 8%)
  adding: kaggle/working/gec-t5-base/special_tokens_map.json (deflated 86%)


In [27]:
from IPython.display import FileLink
FileLink('/kaggle/working/fine-tuned.zip')

/kaggle/working/fine-tuned.zip

In [30]:
# zip trainer-state checkpoint folder to download
!zip  trainer_state.zip /kaggle/working/trainer-state/checkpoint-12000

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


updating: kaggle/working/trainer-state/checkpoint-12000/ (stored 0%)


### Inference

In [33]:
from transformers import pipeline

corrector_model = ("text2text-generation", model="/kaggle/working/gec-t5-base")
text = "Yesterday I go to a supermrket to buy groceries."
correction = corrector(text)
correction[0]['generated_text']

In [41]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/gec-t5-base", model_max_length=1024)
corrector = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/gec-t5-base")

What I did yesterday I don't remember.


In [42]:
sentence = "grammar: One possible outcome are that an environmentally-induced reduction in motorization levels in the richer countrie will outweigh any rise in motorization levels in the poorer countries."
inputs = tokenizer(sentence, return_tensors="pt").input_ids
gen = corrector.generate(inputs, max_new_tokens=1024)
corrected = tokenizer.decode(gen[0], skip_special_tokens=True)
print(corrected)

One possible outcome is that an environmentally-induced reduction in motorization levels in the richer countrie will outweigh any rise in motorization levels in the poorer countrie.


### BLEU Evaluation

In [ ]:
import sacrebleu
import csv
from tqdm import tqdm

data_to_export = []  # List to store data for export
blue_score = 0

for i, sentence in tqdm(enumerate(dataset['test']), total=len(dataset['test'])):
    input_sentence = sentence['input']
    target = sentence['target']
    query = "grammar: "+input_sentence+"output:"
    input_ids = tokenizer(query, return_tensors="pt").input_ids
    outputs = corrector.generate(input_ids, max_length=1024)
    decoder_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    new_blue = sacrebleu.corpus_bleu([decoder_output], [[target]]).score

    blue_score += new_blue
    data_to_export.append((input_sentence, decoder_output, new_blue))  # Add data to the list

blue_score /= len(dataset['test'])

# # Write data to CSV
# with open('inference_bleu_scores.csv', 'w', newline='', encoding='utf-8') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerow(['Input Sentence', 'Inference', 'BLEU Score'])  # Header row
#     writer.writerows(data_to_export)  # Data rows

print("Average BLEU score:", blue_score)